In [40]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer


In [2]:
imp_text = pd.read_csv('imptext.csv')
imp_text.head()

,comment
0,explanation edits username hardcore metallica ...
1,matches background colour im seemingly stuck t...
2,hey man im really trying edit war guy constant...
3,make real suggestions improvement wondered sec...
4,sir hero chance remember page thats


In [3]:
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

train = pd.read_csv('F:/notebook_working/kaggle_compe/train.csv').fillna('No data')
test = pd.read_csv('F:/notebook_working/kaggle_compe/test.csv').fillna('No data')

#train.id = train.id.astype('str')
#train.comment_text = train.comment_text.astype('str')

#test.id = test.id.astype('str')
#test.comment_text = test.comment_text.astype('str')


In [4]:
train_text = train[['id', 'comment_text']]
test_text = test[['id', 'comment_text']]
combined_text = pd.concat([train_text, test_text])

In [24]:
print train_text.shape
print test_text.shape
print train_text.iloc[train_text.shape[0]-1, ]
print test_text.iloc[0, ]
print combined_text.iloc[train_text.shape[0]-1,]
print combined_text.iloc[train_text.shape[0],]

(159571, 2)
(153164, 2)
id                                               fff46fc426af1f9a
comment_text    "\nAnd ... I really don't think you understand...
Name: 159570, dtype: object
id                                               00001cee341fdb12
comment_text    Yo bitch Ja Rule is more succesful then you'll...
Name: 0, dtype: object
id                                               fff46fc426af1f9a
comment_text      really think understand came idea bad right ...
Name: 159570, dtype: object
id                                               00001cee341fdb12
comment_text    yo bitch ja rule succesful ever whats hating s...
Name: 0, dtype: object


In [5]:
combined_text.shape
#combined_text.head()

(312735, 2)

In [6]:
imp_text.comment.isna().sum()

3896

In [7]:
missing = imp_text.loc[imp_text.comment.isna(), ]
missing_index = list(missing.index)


In [8]:
missing_index[:5]

[241L, 258L, 627L, 1050L, 1177L]

In [45]:
#combined_text.iloc[missing_index, ]

In [9]:
#dont use errors='ignore'
combined_text.comment_text = map(lambda x:unicode(x, 'utf-8'), combined_text.comment_text)
#combined_text = map(lambda x:unicode(x, 'utf-8'), combined_text)

In [10]:

#extracting the stopwords from nltk library
sw = stopwords.words('english')
# displaying the stopwords
np.array(sw)

array([u'i', u'me', u'my', u'myself', u'we', u'our', u'ours',
       u'ourselves', u'you', u"you're", u"you've", u"you'll", u"you'd",
       u'your', u'yours', u'yourself', u'yourselves', u'he', u'him',
       u'his', u'himself', u'she', u"she's", u'her', u'hers', u'herself',
       u'it', u"it's", u'its', u'itself', u'they', u'them', u'their',
       u'theirs', u'themselves', u'what', u'which', u'who', u'whom',
       u'this', u'that', u"that'll", u'these', u'those', u'am', u'is',
       u'are', u'was', u'were', u'be', u'been', u'being', u'have', u'has',
       u'had', u'having', u'do', u'does', u'did', u'doing', u'a', u'an',
       u'the', u'and', u'but', u'if', u'or', u'because', u'as', u'until',
       u'while', u'of', u'at', u'by', u'for', u'with', u'about',
       u'against', u'between', u'into', u'through', u'during', u'before',
       u'after', u'above', u'below', u'to', u'from', u'up', u'down',
       u'in', u'out', u'on', u'off', u'over', u'under', u'again',
       u'further'

In [11]:
def stopwords(text):
    '''a function for removing the stopword'''
# removing the stop words and lowercasing the selected words
    text = [word.lower() for word in text.split() if word.lower() not in sw]
    # joining the list of words with space separator
    return " ".join(text)

In [12]:
combined_text['comment_text'] = combined_text['comment_text'].apply(stopwords)
combined_text.head(10)
#combined_text = map(stopwords, combined_text)
#combined_text

,id,comment_text
0,0000997932d777bf,explanation edits made username hardcore metal...
1,000103f0d9cfb60f,d'aww! matches background colour i'm seemingly...
2,000113f07ec002fd,"hey man, i'm really trying edit war. guy const..."
3,0001b41b1c6bb37e,""" can't make real suggestions improvement - wo..."
4,0001d958c54c6e35,"you, sir, hero. chance remember page that's on?"
5,00025465d4725e87,""" congratulations well, use tools well. · talk """
6,0002bcb3da6cb337,cocksucker piss around work
7,00031b1e95af7921,vandalism matt shirvington article reverted. p...
8,00037261f536c51d,"sorry word 'nonsense' offensive you. anyway, i..."
9,00040093b2687caa,alignment subject contrary dulithgow


In [13]:
PunctuationToRemove = [".", ",", ":", ";", "!" ,"?", "&", "\"", "\'", "~", "\\", "="]
def remove_punctuation(text):
    '''a function for removing punctuation'''
    for char in PunctuationToRemove:
        text = text.replace(char,"")
    return text
combined_text['comment_text'] = combined_text['comment_text'].apply(remove_punctuation)
combined_text.head(10)
#combined_text = map(remove_punctuation, combined_text)
#combined_text

,id,comment_text
0,0000997932d777bf,explanation edits made username hardcore metal...
1,000103f0d9cfb60f,daww matches background colour im seemingly st...
2,000113f07ec002fd,hey man im really trying edit war guy constant...
3,0001b41b1c6bb37e,cant make real suggestions improvement - wond...
4,0001d958c54c6e35,you sir hero chance remember page thats on
5,00025465d4725e87,congratulations well use tools well · talk
6,0002bcb3da6cb337,cocksucker piss around work
7,00031b1e95af7921,vandalism matt shirvington article reverted pl...
8,00037261f536c51d,sorry word nonsense offensive you anyway im in...
9,00040093b2687caa,alignment subject contrary dulithgow


In [14]:

missing_values = list(combined_text.iloc[missing_index, 1])
#missing_values

In [15]:
imp_text1 = imp_text.copy()
imp_text1.iloc[missing_index, 0] = missing_values

In [16]:
imp_text1.isna().sum()

comment    0
dtype: int64

In [1]:
#imp_text1.to_csv('F:/notebook_working/imptext1.csv', index=False)

In [26]:
all_text = imp_text1.comment


In [27]:
all_text = list(all_text)
type(all_text)

list

In [29]:
train_text1 = all_text[:train_text.shape[0]]
test_text1 = all_text[train_text.shape[0]:len(all_text)]
print len(train_text1)
print len(test_text1)

159571
153164


In [34]:
#train_text1[len(train_text1)-1]
#test_text1[0]

In [25]:
#all_text

In [47]:
#whole data
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    encoding='utf-8',
    lowercase=True,
    #min_df=0.00001,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    #use_idf=1, smooth_idf=1,
    max_features=None)

word_vectorizer.fit(all_text)
train_word_features = word_vectorizer.transform(train_text.comment_text)
test_word_features = word_vectorizer.transform(test_text.comment_text)

msg = "There are {} tokens in Comment_text if we use word"
print(msg.format(len(word_vectorizer.get_feature_names())))


There are 25393 tokens in Comment_text if we use word


In [37]:
#word_vectorizer.get_feature_names()

In [38]:
#dont run
char_vectorizer = TfidfVectorizer(
    #norm=None,
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    min_df = 0.00001,
    ngram_range=(1,1)) 
    #non_negative=True,
    
    #max_features=10000)
    
char_vectorizer.fit(all_text)
train_char_features = char_vectorizer.transform(train_text1)
test_char_features = char_vectorizer.transform(test_text1)

msg = "There are {} tokens in Comment_text if we use char"
print(msg.format(len(char_vectorizer.get_feature_names())))

There are 854 tokens in Comment_text if we use char


In [41]:
#char_vectorizer.get_feature_names()

In [48]:
#train_features = hstack([train_char_features, train_word_features])
#test_features = hstack([test_char_features, test_word_features])
#X_train, X_valid, y_train, y_valid = train_test_split(train_features, train_target, test_size=0.3, random_state=42)


scores = []
submission = pd.DataFrame.from_dict({'id': test['id']})
for class_name in class_names:
    train_target = train[class_name]
    classifier = LogisticRegression(solver='sag')

    cv_score = np.mean(cross_val_score(classifier, train_word_features, train_target, cv=3, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))

    classifier.fit(train_word_features, train_target)
    #submission[class_name] = classifier.predict_proba(test_word_features)[:, 1]

print('Total CV score is {}'.format(np.mean(scores)))

#submission.to_csv('F:\impwordssub.csv', index=False)

CV score for class toxic is 0.969050842841
CV score for class severe_toxic is 0.985685426799
CV score for class obscene is 0.98543180024
CV score for class threat is 0.978245169467
CV score for class insult is 0.976033206562
CV score for class identity_hate is 0.971681682638
Total CV score is 0.977688021425


In [42]:
#train_features = hstack([train_char_features, train_word_features])
#test_features = hstack([test_char_features, test_word_features])


scores = []
submission = pd.DataFrame.from_dict({'id': test['id']})
for class_name in class_names:
    train_target = train[class_name]
    X_train, X_valid, y_train, y_valid = train_test_split(train_word_features, train_target, test_size=0.3, random_state=42)

    classifier = MultinomialNB()

    cv_score = np.mean(cross_val_score(classifier, X_valid, y_valid, cv=5, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))

    classifier.fit(X_train, y_train)
    #submission[class_name] = classifier.predict_proba(test_features)[:, 1]

print('Total CV score is {}'.format(np.mean(scores)))

#submission.to_csv('submission.csv', index=False)

CV score for class toxic is 0.92123216964
CV score for class severe_toxic is 0.91847880705
CV score for class obscene is 0.921219071701
CV score for class threat is 0.864997542019
CV score for class insult is 0.916600110339
CV score for class identity_hate is 0.906683340467
Total CV score is 0.908201840203
